In [ ]:
#criteria: at least 5 games played in regular season before or after event
#data partially from http://nbafamily.wikia.com/wiki/Category:NBA_Children

player_events=[['J.R. Smith','01/02/2017','birth'], \
               ['J.R. Smith','02/09/2009','birth'], \
               ['Paul Pierce','04/04/2008','birth'], \
               ['Paul Pierce','04/02/2016','birth'], \
               ['Dwyane Wade','02/04/2002','birth'], \
               ['Dwyane Wade','11/10/2013','birth'], \
               ["Shaquille O'Neal",'01/11/2000','birth'], \
               ["Shaquille O'Neal",'11/13/2001','birth'], \
               ["Carmelo Anthony",'03/07/2007','birth'], \
               ["Chris Bosh",'11/04/2013','birth'], \
               ["Kobe Bryant",'01/19/2003','birth'], \
               ["Michael Jordan",'12/07/1992','birth'], \
               ["Michael Jordan",'11/18/1988','birth'], \
               ["Michael Jordan",'12/24/1990','birth'], \
               ['Steve Nash','11/12/2010','birth'], \
               ['Steve Nash','11/12/2010','divorce'], \
               ['Tim Duncan','03/27/2013','divorce'], \
               ["Amar'e Stoudemire",'02/06/2012','death'], \
               ['Mychal Thompson','02/08/1990','child'], \
               ['Tony Parker','11/17/2010','divorce']]

In [1]:
#input player, event date
import requests,bs4
import pandas as pd
from datetime import datetime

#get input of player name and life event date as MM/DD/YYYY
for p_ind,player_info in enumerate(player_events):
player_tofind=player_info[0]
date_lifeevent=player_info[1]

#get player URL
first_letter=player_tofind[player_tofind.find(' ')+1].lower()
url='https://www.basketball-reference.com/players/'+first_letter+'/'

#wrap in url2soup?
res=requests.get(url)
bbsoup=bs4.BeautifulSoup(res.text,'lxml')
body=bbsoup.select('tbody')

names=body[0].find_all('tr')

for player_index,player in enumerate(names):
    if player_tofind==str(player.find_all('a')[0].contents[0]):
        urlnew='https://www.basketball-reference.com/'+player.find_all('a')[0].attrs['href']
        break
    
    #get season URL
    date_month=float(date_lifeevent[0:date_lifeevent.find('/')])
    if date_month>9:
        urlyear=float(date_lifeevent[-4:])+1
    else:
        urlyear=float(date_lifeevent[-4:])
    urlnew=urlnew[0:-5]+'/gamelog/'+str(int(urlyear))+'/'
    
    #get GameScores for specific date range
    gamescores_preevent=pd.DataFrame()
    for pl_seas in samebpm_data.iterrows(): #only the one player/date
        if #between start date and event date
            url='https://www.basketball-reference.com/players/'+str(pl_seas[1][2][0])+'/'+str(pl_seas[1][2])+'/gamelog/'+str(int(pl_seas[1][0]))+'/'
            playerdf=get_gamescores(url)
            pid=pd.Series.repeat(pd.Series(str(pl_seas[1][2])),len(playerdf))
            playerdf['Player ID']=pid.values
            print [str(pl_seas[1][1]),'mean/SD of GameScore:',np.mean(playerdf['GameScore']),np.std(playerdf['GameScore'])]
            gamescores_preevent=gamescores_preevent.append(playerdf)
            
    gamescores_postevent=pd.DataFrame()
    for pl_seas in samebpm_data.iterrows(): #only the one player/date
        if #between start date and event date
            url='https://www.basketball-reference.com/players/'+str(pl_seas[1][2][0])+'/'+str(pl_seas[1][2])+'/gamelog/'+str(int(pl_seas[1][0]))+'/'
            playerdf=get_gamescores(url)
            pid=pd.Series.repeat(pd.Series(str(pl_seas[1][2])),len(playerdf))
            playerdf['Player ID']=pid.values
            print [str(pl_seas[1][1]),'mean/SD of GameScore:',np.mean(playerdf['GameScore']),np.std(playerdf['GameScore'])]
            gamescores_postevent=gamescores_postevent.append(playerdf)
            
    

207


'https://www.basketball-reference.com//players/j/jordami01/gamelog/1993/'

In [3]:
def get_gamescores(url):
    #get GameScores from given Basketball Reference URL
    
    import requests,bs4,pandas as pd
    
    dftemp=[]
    res=requests.get(url)
    bbsoup=bs4.BeautifulSoup(res.text,'lxml')
    body=bbsoup.select('tbody')
    
    games=body[0].find_all('tr')
    for game_index,game in enumerate(games):
        stats=game.find_all('td')
        
        for stat_index,stat in enumerate(stats):
            if (stat.attrs['data-stat']=='game_score') & (len(stat)>0):
                dftemp.append([str(stats[1].contents[0].contents[0]), float(stats[stat_index].contents[0])])
    df=pd.DataFrame(dftemp,columns=['Date','GameScore'])
    return df

In [9]:
# #playoff stats in different table, unparseable in comment
# playoffs=bbsoup.find_all("div",  id="all_pgl_basic_playoffs")
#all below code is just getting original table again
# playoffsmaybe=bbsoup.find_all("div", attrs ={"class":"table_outer_container"})
# playofftable=playoffsmaybe[0].contents[1].contents[1]
# column_headers = [th.getText() for th in playofftable.thead.tr.find_all('th')]
# 
# #get indices
# for index,value in enumerate(column_headers):
#     if value=='GmSc':
#         gsc_index=index
#     elif value=='Date':
#         date_index=index
# 
# 
# games=playofftable.tbody.find_all('tr')
# plgsc=[]
# for game_index,game in enumerate(games):
#     stats=game.find_all('td')
#     
#     for stat_index,stat in enumerate(stats):
#         if (stat.attrs['data-stat']=='game_score') & (len(stat)>0):
#             plgsc.append([str(stats[1].contents[0].contents[0]),float(stats[stat_index].contents[0])])


In [ ]:

# is month before life-event (MBLE) different from player-season distribution

# is MPLE different from month after life event (MALE)?


In [1]:
# get season BPM for player

# find similar player-seasons within 10% of player's season BPM

# using "Historical" sheet from Box Plus Minus 1.1 Historical for BBRef.xlsm 
# from https://www.basketball-reference.com/about/bpm.html

import pandas as pd
xl = pd.ExcelFile("BPM_1_1_Historical_BBRef.xls")
df = xl.parse("Sheet1")
player_tofind='LeBron James'
year_tofind=2010
playerslice=df[(df['Player']==player_tofind) & (df['Year']==year_tofind)]
player_bpm=float(playerslice['BPM'])
minBPM=player_bpm*0.8
maxBPM=player_bpm*1.2
samebpm=df[(df['BPM']>minBPM) & (df['BPM']<maxBPM)]
samebpm_data=samebpm[['Year','Player','Player ID','BPM','G']]

#floor of 15 games that season to compare appropriate sample sizes
samebpm_data=samebpm_data[samebpm_data['G']>15]



In [4]:
# record distribution of GameScores for those player-seasons

import numpy as np
gamescores_all=pd.DataFrame()
for pl_seas in samebpm_data.iterrows():
    if not((pl_seas[1][1] == player_tofind) & (pl_seas[1][0] == year_tofind)):
        url='https://www.basketball-reference.com/players/'+str(pl_seas[1][2][0])+'/'+str(pl_seas[1][2])+'/gamelog/'+str(int(pl_seas[1][0]))+'/'
        playerdf=get_gamescores(url)
        pid=pd.Series.repeat(pd.Series(str(pl_seas[1][2])),len(playerdf))
        playerdf['Player ID']=pid.values
        print [str(pl_seas[1][1]),'mean/SD of GameScore:',np.mean(playerdf['GameScore']),np.std(playerdf['GameScore'])]
        gamescores_all=gamescores_all.append(playerdf)
        
print ['Overall mean/SD: ',np.mean(gamescores_all['GameScore']),np.std(gamescores_all['GameScore'])]


['Michael Jordan', 'mean/SD of GameScore:', 28.813414634146344, 8.8362855317610833]


['Michael Jordan', 'mean/SD of GameScore:', 28.595061728395059, 8.1507994190158488]


['Michael Jordan', 'mean/SD of GameScore:', 27.570731707317076, 9.5524545154364606]


['Michael Jordan', 'mean/SD of GameScore:', 26.213414634146343, 7.0233888386802299]


['David Robinson', 'mean/SD of GameScore:', 25.818750000000001, 9.2091950482927665]


...

['LeBron James', 'mean/SD of GameScore:', 24.405263157894741, 6.0841445088387029]


['Stephen Curry', 'mean/SD of GameScore:', 20.386249999999997, 7.6935158372164283]


['Russell Westbrook', 'mean/SD of GameScore:', 21.565671641791045, 9.0909755521173761]


['Russell Westbrook', 'mean/SD of GameScore:', 20.887499999999999, 7.4591282164875006]


['Stephen Curry', 'mean/SD of GameScore:', 24.579746835443036, 8.6776503228570547]
['Overall mean/SD: ', 24.571048109965634, 8.5412791038370948]
